In [ ]:
# WSL
# cd /mnt/c/Users/rs659/Desktop/mmsearch

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from inference import inference
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = PeftConfig.from_pretrained("HongxuanLi/TinyLLaMA-RS")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = PeftModel.from_pretrained(model, "HongxuanLi/TinyLLaMA-RS").to(device)
tokenizer = AutoTokenizer.from_pretrained( "TinyLlama/TinyLlama-1.1B-Chat-v1.0", add_bos_token=True, trust_remote_code=True)

In [ ]:
result = inference.inference(model,tokenizer,'Give me my latest resume')

In [ ]:
result

In [ ]:
result['file content']

In [ ]:
# conda create -n mmsearchenv python=3.9
# conda activate mmsearchenv
# conda install pytorch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 pytorch-cuda=12.1 -c pytorch -c nvidia
# pip install -r requirements.txt
# docker-compose up -d

# https://anaconda.org/conda-forge/docker-compose



# docker rm -f $(docker ps -aq)


# conda install -c conda-forge poppler


# Windows
# cd  C:\Users\rs659\Desktop\Multi-Modal-Local-File-Search-Engine
# conda deactivate
# conda activate mmsearchenv
# docker-compose up -d
# python add_data.py
# streamlit run app.py



# WSL
# cd /mnt/c/Users/rs659/Desktop/Multi-Modal-Local-File-Search-Engine
# conda activate mmsearchenv
# docker-compose up -d
# python add_data.py
# streamlit run app.py


# https://github.com/weaviate/weaviate-examples/blob/main/weaviate-transformers-newspublications/docker-compose-withgpu.yaml
# https://www.pexels.com/search/videos/cat/

In [ ]:
# Add metadata of PDF
# Add progress to batch insert

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
# https://developer.nvidia.com/cuda-12-2-0-download-archive?target_os=Linux&target_arch=x86_64&Distribution=WSL-Ubuntu&target_version=2.0&target_type=deb_local

In [ ]:
# https://github.com/weaviate-tutorials/DEMO-multimodal-search

In [ ]:
from pathlib import Path
import json

for f in Path("data/podcasts").glob("*.json"):  # assuming podcast data is in JSON files
    # podcast_data = json.loads(f.read_text())
    with open(f, 'r', encoding='utf-8') as file:
        podcast_data = json.load(file)

In [ ]:
len(podcast_data)

In [ ]:
# data_props = {
#     "title": podcast_data["title"],
#     "transcript": podcast_data["transcript"],
#     "description": podcast_data["description"],
#     "titleimage": base64.b64encode(Path(podcast_data["titleimage"]).read_bytes()).decode(),
# }

# Create PDF Object

In [ ]:
import weaviate
import weaviate.classes as wvc
from weaviate.util import generate_uuid5
from weaviate import WeaviateClient
from weaviate.collections.classes.batch import BatchObjectReturn
import base64
from pathlib import Path
import pandas as pd
import json
from unstructured.partition.pdf import partition_pdf
from AbstractExtractor import AbstractExtractor

In [ ]:
def define_collection_pdfs(client: WeaviateClient, collection_name: str = 'pdfs') -> bool:

    # Creating a new collection with the defined schema
    client.collections.create(
        name=collection_name,
        properties=[
            wvc.Property(
                name="filename",
                data_type=wvc.DataType.TEXT,
            ),
            wvc.Property(
                name="abstract",
                data_type=wvc.DataType.TEXT,
            ),
            wvc.Property(
                name="pages",
                # data_type=wvc.DataType.OBJECT_ARRAY,
                data_type=wvc.DataType.TEXT_ARRAY,
            )
        ],
        vectorizer_config=wvc.config.Configure.Vectorizer.multi2vec_bind(
            text_fields=[wvc.config.Multi2VecField(name='filename', weight=0.50),
                         wvc.config.Multi2VecField(name='abstract', weight=0.50)],
            vectorize_collection_name=False)
    )

In [ ]:
client = weaviate.connect_to_local()
client.collections.delete('pdf')

define_collection_pdfs(client,'pdf')
# import_data_pdf(client)

In [ ]:
data_folder = "data/pdf/"

data_objects = []

for path in Path(data_folder).iterdir():
    if path.suffix != ".pdf":
        continue

    print(f"Processing {path.name}...")

    elements = partition_pdf(filename=path)
    break
    # abstract_extractor = AbstractExtractor()
    # abstract_extractor.consume_elements(elements)

    # data_object = {"filename": path.name, "abstract": abstract_extractor.abstract()[:50], "pages":[data.text for data in elements]}

    # data_objects.append(data_object)

# Pdfs

In [ ]:
from create_collections.PDF import define_collection_pdfs, import_data_pdf
from pathlib import Path
import weaviate
import weaviate.classes as wvc
import streamlit as st
import base64
from unstructured.partition.pdf import partition_pdf
from AbstractExtractor import AbstractExtractor

In [ ]:
client = weaviate.connect_to_local()

In [ ]:
pdf_collection = client.collections.get('pdf')

# pdf_collection.data.insert_many(data_objects)

In [ ]:
from weaviate.classes.query import MetadataQuery

jeopardy = client.collections.get("pdf")
response = jeopardy.query.bm25(
    query="car safety is required",
    query_properties=["pages"],
    return_metadata=MetadataQuery(score=True),
    limit=3
)


for o in response.objects:
    print(o)

In [ ]:
jeopardy = client.collections.get("pdf")
response = jeopardy.query.hybrid(
    query="car safety is required",
    alpha=0.5,
    return_metadata=MetadataQuery(score=True, explain_score=True),
    limit=3
)

for o in response.objects:
    print(o)


In [ ]:
response = pdf_collection.query.fetch_objects(
    include_vector=True,
    limit=2
)

for o in response.objects:
    print(o)

In [ ]:
o

In [ ]:
response = pdf_collection.query.bm25(
     query="price",
    limit=3
)

for o in response.objects:
    print(o.properties)

In [ ]:
response = pdf_collection.query.hybrid(
     query="car",
    limit=3
)

for o in response.objects:
    print(o.properties)

# Check reponse

In [1]:
from pathlib import Path
import weaviate
import weaviate.classes as wvc
import streamlit as st
import base64
# from add_data import COLLECTION_NAME

client = weaviate.connect_to_local()

In [11]:
pdf_collection = client.collections.get('pdf')

response = pdf_collection.aggregate.over_all(total_count=True)
print(f"Object count in the Database: {response.total_count}")

for q in ["resume"]:
    response = pdf_collection.query.bm25(
            q, 
            limit=12,
            include_vector=False,
            # distance=0.5,
            return_properties=['filename','date_modified','date_created']
            )

print(response)
print(len(response.objects))

# big_reponse_list = []
# big_reponse_list.append(response.objects)

Object count in the Database: 17
_QueryReturn(objects=[_Object(uuid=_WeaviateUUIDInt('c26beb1d-382a-4b4f-b65d-ef7150b6f348'), metadata=_MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None), properties={'date_created': datetime.datetime(2024, 4, 16, 15, 57, 21, 706756, tzinfo=datetime.timezone.utc), 'filename': 'Resume_Zihan_Xing.pdf', 'date_modified': datetime.datetime(2024, 2, 14, 4, 8, 48, 493160, tzinfo=datetime.timezone.utc)}, references=None, vector=None), _Object(uuid=_WeaviateUUIDInt('8fae554a-226e-4402-b0d0-5c382a53e564'), metadata=_MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None), properties={'date_created': datetime.datetime(2024, 4, 16, 15, 57, 21, 708419, tzinfo=datetime.timezone.utc), 'filename': 'Resume Shuaiming Jing.pdf', 'date_modified': datetime.datetime(2023, 9, 7, 23, 3, 5, 554652, tzinfo=

In [12]:
[obj.properties for obj in response.objects]

[{'date_created': datetime.datetime(2024, 4, 16, 15, 57, 21, 706756, tzinfo=datetime.timezone.utc),
  'filename': 'Resume_Zihan_Xing.pdf',
  'date_modified': datetime.datetime(2024, 2, 14, 4, 8, 48, 493160, tzinfo=datetime.timezone.utc)},
 {'date_created': datetime.datetime(2024, 4, 16, 15, 57, 21, 708419, tzinfo=datetime.timezone.utc),
  'filename': 'Resume Shuaiming Jing.pdf',
  'date_modified': datetime.datetime(2023, 9, 7, 23, 3, 5, 554652, tzinfo=datetime.timezone.utc)},
 {'filename': 'Resume_Zihan_Zach_Xing.pdf',
  'date_created': datetime.datetime(2024, 4, 16, 15, 57, 21, 707071, tzinfo=datetime.timezone.utc),
  'date_modified': datetime.datetime(2024, 1, 25, 23, 17, 49, 650065, tzinfo=datetime.timezone.utc)}]

In [ ]:
# response = pdf_collection.aggregate.over_all(total_count=True)
# response